In [13]:
import cv2
import cvzone
import sys
from cvzone.HandTrackingModule import HandDetector
import numpy as np


cap = cv2.VideoCapture(1)
cap.set(3, 2560)
cap.set(4, 1440)
detector = HandDetector(detectionCon=1)
colorR = (255, 0, 255)
startdist=None
scale=0


cx, cy, w, h = 100, 100, 200, 200


class DragRect():
    def __init__(self, posCenter, size=[200, 200]):
        self.posCenter = posCenter
        self.size = size

    def update(self, cursor):
        cx, cy = self.posCenter
        w, h = self.size

        # If the index finger tip is in the rectangle region
        if cx - w // 2 < cursor[0]< cx + w // 2 and \
                cy - h // 2 < cursor[1]< cy + h // 2:
            self.posCenter = cursor
            
    def zoom(self, cursor,scale):
        cx, cy = self.posCenter
        w, h = self.size

        # If the index finger tip is in the rectangle region
        if cx - w // 2 < cursor[0]< cx + w // 2 and \
                cy - h // 2 < cursor[1]< cy + h // 2:
            self.posCenter = cursor
            if 100< w+scale <600:
                self.size=[w+scale,h+scale]
    def resize(self,cursor):
        cx, cy = self.posCenter
        w, h = self.size
        if cx - w // 2 < cursor[0]< cx + w // 2 and \
                cy - h // 2 < cursor[1]< cy + h // 2:
            self.size=[200,200]
                


rectList = []
for x in range(5):
    rectList.append(DragRect([x * 250 + 150, 150]))





In [14]:
while True:
    success, img = cap.read()
    img1 = cv2.flip(img, 1)
    lmList = detector.findHands(img1,draw=False)
    
    


    if lmList:
        for hand in lmList:
            _,cursor= detector.findDistance(hand["lmList"][9], hand["lmList"][13])
            if   detector.fingersUp(hand)==[1,1,1,1,1] :
                # call the update here
                for rect in rectList:
                    rect.update(cursor[4:])
            if  detector.fingersUp(hand)==[1,1,1,0,0] :
                for rect in rectList:
                    rect.resize(cursor[4:])
            
    if len(lmList)==2:
        hand1=lmList[0]
        hand2=lmList[1]
        if detector.fingersUp(hand1)==[1,1,0,0,0] and detector.fingersUp(hand2)==[1,1,0,0,0]:
            l1,info=detector.findDistance(hand1["lmList"][8],hand2["lmList"][8])    
            if startdist is None:
                startdist=l1
            scale=int((l1-startdist)//2)
            for rect in rectList:
                    rect.zoom(info[4:],scale)

        else :
            startdist=None


    imgNew = np.zeros_like(img1, np.uint8)
    for rect in rectList:
        cx, cy = rect.posCenter
        w, h = rect.size
        cv2.rectangle(imgNew, (cx - w // 2, cy - h // 2),
                      (cx + w // 2, cy + h // 2), colorR, cv2.FILLED)
        cvzone.cornerRect(imgNew, (cx - w // 2, cy - h // 2, w, h), 20, rt=0)

    out = img1.copy()
    alpha = 0.5
    mask = imgNew.astype(bool)
    out[mask] = cv2.addWeighted(img1, alpha, imgNew, 1 - alpha, 0)[mask]

    cv2.imshow("Image", out)
    if cv2.waitKey(27)==27:
        cap.release()
        cv2.destroyAllWindows()
        
        break
        